# Estimation of Demand for Beer using PyBLP

In [1]:
import pyblp
import os
import numpy as np
import pandas as pd
import warnings
import math
import copy
from pandas.core.common import SettingWithCopyWarning
import matplotlib.pyplot as plt
import tabulate as tabulate

In [2]:
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
pyblp.options.digits = 4
pyblp.options.verbose = False

This code estimates the demand model for beer following the specifications in [Miller and Weinberg (2017)](https://onlinelibrary.wiley.com/doi/abs/10.3982/ECTA13333) and IRI scanner dataset. We use the RCNL-1 specification where income is interacted with price, the inside good constant, and calories. The unit of observation for this analysis is at the brand-size-region-year-month level. 

The following steps load the data and set up the problem:
1. Set the path to the working directory.
2. Load the product data.
3. Generate instrumental variables.
4. Add demographic data (income)


In [3]:
# 1. Set the path to load the datasets
path = ('XXX')

In [4]:
# 2. Load the monthly product data (id,price,share,attributes,mkt size, etc)
def prep_data():
    data = pd.read_csv(path + 'data\\blp_beer_revised_final.csv')
    data['firm_ids']=np.floor(data['blpid']/10000000000)
    data['brand_ids']=np.floor((data['blpid']-data['firm_ids']*10000000000)/100000000)
    data['size_ids'] = np.floor((data['blpid']-data['firm_ids']*10000000000-data['brand_ids']*100000000)/1000000)
    data['city_ids']=np.floor((data['blpid']-data['firm_ids']*10000000000-data['brand_ids']*100000000-data['size_ids']*1000000)/10000)
    data['year']=np.floor((data['blpid']-data['firm_ids']*10000000000-data['brand_ids']*100000000-data['size_ids']*1000000-data['city_ids']*10000)/100)
    data['month']=np.floor((data['blpid']-data['firm_ids']*10000000000-data['brand_ids']*100000000-data['size_ids']*1000000-data['city_ids']*10000 - data['year']*100))
    #Products are brand x pack size combinations
    data['product_ids']= 'B'+data['brand_ids'].astype(str)+'S'+data['size_ids'].astype(str)
    data['market_ids'] = 'C'+data['city_ids'].astype(str)+'Y'+data['year'].astype(str)+'M'+data['month'].astype(str)
    data['clustering_ids']='C'+data['city_ids'].astype(str)    
    data['date'] = data['year'].astype(str)+'/'+data['month'].astype(str)
    data.rename(columns={'real_price':'prices'},inplace=True)
    data['index'] = data.index #Unique identifier of obsevartion    

    iv = pd.read_csv(path + 'data\\blp_iv_revised_final.csv')
    iv['index'] = iv.index #Unique identifier of obsevartion   
    temp = pd.merge(data,iv,on='index') #Both datasets are generated from the same database and sorted similarly (from stata). This avoids the duplicates issue
    
    #Observations included -- exclude year after merger approved
    temp = temp.loc[((temp.year <= 3)|(temp.year >= 6)|((temp.year == 4)&(temp.month < 6)) | ((temp.year == 5)&(temp.month >= 6)))]
    s=temp.shape[0]
    print('====After excluding year after merge approved, sample size is '+str(s)+" ====")
    temp['calories_144']=temp['calories_144']/100
    temp['calories_144'] = temp['calories_144'] -temp['calories_144'].mean()
    temp['import_d'] = temp['brand_ids'].isin([5,6,7,8]).astype(int)

    #Reassign Inbev to Budweiser
    temp.loc[temp.firm_ids==4,'firm_ids']=1
    temp['bmc'] = 0
    temp.loc[temp.firm_ids.isin([1,5,6]),'bmc']=1
    temp['bud'] = 0
    temp.loc[temp.firm_ids ==1,'bud']=1

    #calculate shares
    temp['mktsize']=(temp['maxsize_all_10']/1.10)*1.50
    temp['shares']=temp['equivalent_units_144oz']/temp['mktsize']    
    return temp    

In [5]:
# 3. Load additional data and generate instruments for Nested Logit (NL)
def prod_w_iv_nl(df_prod_input): 
    df_prod = df_prod_input.copy()
    groups = df_prod.groupby(['market_ids'])
    #number of product in each market
    df_prod['nprod'] = groups['shares'].transform(np.size)
    #total distance
    df_prod['tdist'] = groups['diesel_dist'].transform('sum')

    #Create more instruments
    df_prod['bmc_tdist'] = df_prod['bmc']*df_prod['tdist']
    df_prod['bmc_nprod'] = df_prod['bmc']*df_prod['nprod']
    df_prod['bud_tdist'] = df_prod['bud']*df_prod['tdist']
    df_prod['bud_nprod'] = df_prod['bud']*df_prod['nprod']
    #df_prod.head()

    #Demand instruments
    df_prod['demand_instruments0'] = df_prod['bmc2008']
    df_prod['demand_instruments1'] = df_prod['diesel_dist']
    df_prod['demand_instruments2'] = df_prod['tdist']
    df_prod['demand_instruments3'] = df_prod['nprod']
    df_prod['demand_instruments4'] = df_prod['bmc']*df_prod['tdist']
    df_prod['demand_instruments5'] = df_prod['bmc']*df_prod['nprod']
    df_prod['demand_instruments6'] = df_prod['bud']*df_prod['tdist']
    df_prod['demand_instruments7'] = df_prod['bud']*df_prod['nprod']   
    return df_prod     

In [6]:
# 3. Load additional data and generate instruments for RCNL-1 (RCNL-1)
def prod_w_iv(df_prod_input): 
    df_prod = df_prod_input.copy()
    groups = df_prod.groupby(['market_ids'])
    #number of product in each market
    df_prod['nprod'] = groups['shares'].transform(np.size)
    #total distance
    df_prod['tdist'] = groups['diesel_dist'].transform('sum')

    df1 = pd.read_csv(path + 'data\\demosSum.csv')
    d = {2005: 1, 2006: 2, 2007: 3, 2008: 4,2009:5, 2010:6, 2011:7}
    df1['year'] = df1['year'].map(d)
    df1.rename(columns={'market':'city_ids'},inplace=True)
    df2 = pd.merge(df_prod,df1,on=['year','city_ids'])

    #Demand instruments
    df2['demand_instruments0'] = df2['bmc2008']
    df2['demand_instruments1'] = df2['diesel_dist']
    df2['demand_instruments2'] = df2['tdist']
    df2['demand_instruments3'] = df2['nprod']
    df2['demand_instruments4'] = df2['bmc']*df2['tdist']
    df2['demand_instruments5'] = df2['bmc']*df2['nprod']
    df2['demand_instruments6'] = df2['bud']*df2['tdist']
    df2['demand_instruments7'] = df2['bud']*df2['nprod']
    df2['demand_instruments8'] = df2['minc']
    df2['demand_instruments9'] = df2['minc']*df2['import_d']
    df2['demand_instruments10'] = df2['minc']*df2['calories_144']
    df2['demand_instruments11'] = df2['minc']*df2['packsize']
    return df2     

In [7]:
# 4. Add demographics (income) to the problem RCNL-1
def prep_agent_data(product_data):
    df1 = pd.read_csv(path + 'data\\demos.csv')
    df_income=df1.loc[:, df1.columns.str.startswith("realInc")|df1.columns.str.startswith("year")|df1.columns.str.startswith("market")]
    year_list = df_income['year'].tolist()
    year_list = list(dict.fromkeys(year_list))
    mkt_list = df_income['market'].tolist()
    mkt_list = list(dict.fromkeys(mkt_list))

    agent_data = pd.DataFrame()
    for y in year_list:
        for m in mkt_list:
            temp = df_income[(df_income.year==y)&(df_income.market==m)]
            temp.drop(columns=['year', 'market'],inplace=True)
            temp_inv = temp.T
            temp_inv.rename(columns={ temp_inv.columns[0]: "income" }, inplace = True)
            temp_inv['year']=y
            temp_inv['city_ids']=m
            agent_data = agent_data.append(temp_inv)

    s=agent_data.shape[0]        
    print('==== Number of rows (years(7)*mkts(39)*individuals(500)) should be '+str(7*39*500)+'-->'+str(s)+" ====")

    temp = agent_data.reset_index().drop(columns=['index'])    
    d = {2005: 1, 2006: 2, 2007: 3, 2008: 4,2009:5, 2010:6, 2011:7}
    temp['year'] = temp['year'].map(d)
    temp1 = product_data[['year','city_ids','market_ids']].drop_duplicates()
    agent_data = pd.merge(temp,temp1,on=['year','city_ids'])
    groups = agent_data.groupby(['market_ids'])
    agent_data['weights'] = 1./ groups['income'].transform(np.size)
    #demean agent income
    agent_data['income'] = agent_data['income']- agent_data['income'].mean()    #MW demeans income slightly different -> should not matter

    mean = [0, 0,0] 
    matrix = [[1, 0,0], [0, 1, 0],[0,0,1]] 
    gfg = np.random.multivariate_normal(mean, matrix, agent_data.shape[0]) 
    nodes = pd.DataFrame(gfg,columns=['nodes0','nodes1','nodes2'])

    return pd.concat([agent_data, nodes], axis=1)

## Estimation 

We estimate the demand model using the standard two-step Generalized Method of Moments (GMM). We use the following options to solve the problem:

- The optimization algorithm employed is trust-constr.  
- For the contraction mapping, we use the 'safe_nonlinear' option.
- We set bounds for the parameters so the routine does not try out unreasonable large parameter values that might create numerical issues.
- We do not center the sample moments before updating the weighting matrix. 
- In the second step, we update the (optimal) weighting matrix by using the cluster correction to correct for heteroscedasticity, autocorrelation, and within-market correlations. 
- Standard errors are clustered by region (market).



### Analysis of Nested Logit (NL)

#### 1. Estimation of demand model

In [8]:
# NL1 (Method = 1s; W Clustered = 0 ; Centered Moments = 0 )
# Starting values
rho_start = 0.5

# Formulation for the linear and nonlinear parts of the model. 
def solve_nl(df_prod):
    df_prod['nesting_ids'] = 1
    nl_formulation = pyblp.Formulation('0 + prices', absorb='C(product_ids)+C(date)')
    problem = pyblp.Problem(nl_formulation, df_prod)
    results = problem.solve(rho=rho_start,se_type='clustered',method='1s')
    return results

# Solve the problem NL
data1 = prep_data()
data_product_iv = prod_w_iv_nl(data1)
NL_results1 = solve_nl(data_product_iv)

# Compute economic stats
elasticities_NL1 = NL_results1.compute_elasticities()
diversions_NL1 = NL_results1.compute_diversion_ratios()
Own_Price_Elasticity_NL1 = NL_results1.extract_diagonals(elasticities_NL1)
Median_Own_Price_Elasticity_NL1 = np.quantile(Own_Price_Elasticity_NL1, 0.5)
Outside_Diversion_NL1 = NL_results1.extract_diagonals(diversions_NL1)
Median_Outside_Diversion_NL1 = np.quantile(Outside_Diversion_NL1, 0.5)
aggregates_NL1 = NL_results1.compute_aggregate_elasticities(factor=0.1)
Median_agg_demand_elas_NL1 = np.quantile(aggregates_NL1, 0.5)
#Vector with cross-elasticity to outside good
ielas_NL1 = Own_Price_Elasticity_NL1 * Outside_Diversion_NL1
Median_Market_Price_Elasticity_NL1 = np.quantile(ielas_NL1, 0.5)

====After excluding year after merge approved, sample size is 94656 ====


In [9]:
# NL2 (Method = 2s; W Clustered = 0 ; Centered Moments = 0 )

# Formulation for the linear and nonlinear parts of the model. 
def solve_nl(df_prod):
    df_prod['nesting_ids'] = 1
    nl_formulation = pyblp.Formulation('0 + prices', absorb='C(product_ids)+C(date)')
    problem = pyblp.Problem(nl_formulation, df_prod)
    results = problem.solve(rho=rho_start,se_type='clustered',method='2s',center_moments=False)
    return results

# Solve the problem NL
data1 = prep_data()
data_product_iv = prod_w_iv_nl(data1)
NL_results = solve_nl(data_product_iv)
NL_results2 = solve_nl(data_product_iv)

# Compute economic stats
elasticities_NL2 = NL_results2.compute_elasticities()
diversions_NL2 = NL_results2.compute_diversion_ratios()
Own_Price_Elasticity_NL2 = NL_results2.extract_diagonals(elasticities_NL2)
Median_Own_Price_Elasticity_NL2 = np.quantile(Own_Price_Elasticity_NL2, 0.5)
Outside_Diversion_NL2 = NL_results2.extract_diagonals(diversions_NL2)
Median_Outside_Diversion_NL2 = np.quantile(Outside_Diversion_NL2, 0.5)
aggregates_NL2 = NL_results2.compute_aggregate_elasticities(factor=0.1)
Median_agg_demand_elas_NL2 = np.quantile(aggregates_NL2, 0.5)
#Vector with cross-elasticity to outside good
ielas_NL2 = Own_Price_Elasticity_NL2 * Outside_Diversion_NL2
Median_Market_Price_Elasticity_NL2 = np.quantile(ielas_NL2, 0.5)

====After excluding year after merge approved, sample size is 94656 ====


In [10]:
# NL3 (Method = 2s; W Clustered = 1 ; Centered Moments = 0 )

# Formulation for the linear and nonlinear parts of the model. 
def solve_nl(df_prod):
    df_prod['nesting_ids'] = 1
    nl_formulation = pyblp.Formulation('0 + prices', absorb='C(product_ids)+C(date)')
    problem = pyblp.Problem(nl_formulation, df_prod)
    results = problem.solve(rho=rho_start,se_type='clustered',W_type='clustered',method='2s',center_moments=False)
    return results

# Solve the problem NL
data1 = prep_data()
data_product_iv = prod_w_iv_nl(data1)
NL_results = solve_nl(data_product_iv)
NL_results3 = solve_nl(data_product_iv)

# Compute economic stats
elasticities_NL3 = NL_results3.compute_elasticities()
diversions_NL3 = NL_results3.compute_diversion_ratios()
Own_Price_Elasticity_NL3 = NL_results3.extract_diagonals(elasticities_NL3)
Median_Own_Price_Elasticity_NL3 = np.quantile(Own_Price_Elasticity_NL3, 0.5)
Outside_Diversion_NL3 = NL_results3.extract_diagonals(diversions_NL3)
Median_Outside_Diversion_NL3 = np.quantile(Outside_Diversion_NL3, 0.5)
aggregates_NL3 = NL_results3.compute_aggregate_elasticities(factor=0.1)
Median_agg_demand_elas_NL3 = np.quantile(aggregates_NL3, 0.5)
#Vector with cross-elasticity to outside good
ielas_NL3 = Own_Price_Elasticity_NL3 * Outside_Diversion_NL3
Median_Market_Price_Elasticity_NL3 = np.quantile(ielas_NL3, 0.5)

====After excluding year after merge approved, sample size is 94656 ====


In [11]:
# NL4 (Method = 2s; W Clustered = 1 ; Centered Moments = 1 )

# Formulation for the linear and nonlinear parts of the model. 
def solve_nl(df_prod):
    df_prod['nesting_ids'] = 1
    nl_formulation = pyblp.Formulation('0 + prices', absorb='C(product_ids)+C(date)')
    problem = pyblp.Problem(nl_formulation, df_prod)
    results = problem.solve(rho=rho_start,se_type='clustered',W_type='clustered',method='2s',center_moments=True)
    return results

# Solve the problem NL
data1 = prep_data()
data_product_iv = prod_w_iv_nl(data1)
NL_results = solve_nl(data_product_iv)
NL_results4 = solve_nl(data_product_iv)

# Compute economic stats
elasticities_NL4 = NL_results4.compute_elasticities()
diversions_NL4 = NL_results4.compute_diversion_ratios()
Own_Price_Elasticity_NL4 = NL_results4.extract_diagonals(elasticities_NL4)
Median_Own_Price_Elasticity_NL4 = np.quantile(Own_Price_Elasticity_NL4, 0.5)
Outside_Diversion_NL4 = NL_results4.extract_diagonals(diversions_NL4)
Median_Outside_Diversion_NL4 = np.quantile(Outside_Diversion_NL4, 0.5)
aggregates_NL4 = NL_results4.compute_aggregate_elasticities(factor=0.1)
Median_agg_demand_elas_NL4 = np.quantile(aggregates_NL4, 0.5)
#Vector with cross-elasticity to outside good
ielas_NL4 = Own_Price_Elasticity_NL4 * Outside_Diversion_NL4
Median_Market_Price_Elasticity_NL4 = np.quantile(ielas_NL4, 0.5)

====After excluding year after merge approved, sample size is 94656 ====


In [12]:
# NL5 (Method = 2s; W Clustered = 0 ; Centered Moments = 1 )

# Formulation for the linear and nonlinear parts of the model. 
def solve_nl(df_prod):
    df_prod['nesting_ids'] = 1
    nl_formulation = pyblp.Formulation('0 + prices', absorb='C(product_ids)+C(date)')
    problem = pyblp.Problem(nl_formulation, df_prod)
    results = problem.solve(rho=rho_start,se_type='clustered',method='2s',center_moments=True)
    return results

# Solve the problem NL
data1 = prep_data()
data_product_iv = prod_w_iv_nl(data1)
NL_results = solve_nl(data_product_iv)
NL_results5 = solve_nl(data_product_iv)

# Compute economic stats
elasticities_NL5 = NL_results5.compute_elasticities()
diversions_NL5 = NL_results5.compute_diversion_ratios()
Own_Price_Elasticity_NL5 = NL_results5.extract_diagonals(elasticities_NL5)
Median_Own_Price_Elasticity_NL5 = np.quantile(Own_Price_Elasticity_NL5, 0.5)
Outside_Diversion_NL5 = NL_results5.extract_diagonals(diversions_NL5)
Median_Outside_Diversion_NL5 = np.quantile(Outside_Diversion_NL5, 0.5)
aggregates_NL5 = NL_results5.compute_aggregate_elasticities(factor=0.1)
Median_agg_demand_elas_NL5 = np.quantile(aggregates_NL5, 0.5)
#Vector with cross-elasticity to outside good
ielas_NL5 = Own_Price_Elasticity_NL5 * Outside_Diversion_NL5
Median_Market_Price_Elasticity_NL5 = np.quantile(ielas_NL5, 0.5)

====After excluding year after merge approved, sample size is 94656 ====


#### 2. Store the output

In [85]:
# Demand estimates in ECMA paper and Matlab
ecma_price_coef = -0.1312
ecma_price_se =  0.0884
ecma_nesting_coef = 0.6299
ecma_nesting_se =  0.0941
ecma_elast_own = -3.81
ecma_elast_mkt = -1.10
ecma_out_diversion = round(0.2980*100,2)

matlab_price_coef = -0.1243
matlab_price_se =  0.0903
matlab_nesting_coef = 0.6349
matlab_nesting_se =  0.0909
matlab_elast_own = round(-3.6585,2)
matlab_elast_mkt = round(-1.0402,2)
matlab_out_diversion = round(0.2949*100,2)

# NL1 (Method = 1s;W Clustered = 0;Centered Moments = 0)
Obs = data1.shape[0]
NL1_price_coef = round(float(NL_results1.beta[0]),4)
NL1_price_se =  round(float(NL_results1.beta_se[0]),4)
NL1_nesting_coef = round(float(NL_results1.rho),4)
NL1_nesting_se = round(float(NL_results1.rho_se),4)
NL1_elast_own = round(float(Median_Own_Price_Elasticity_NL1),2)
NL1_elast_mkt =round(float(Median_Market_Price_Elasticity_NL1),2)
NL1_out_diversion = round(float(Median_Outside_Diversion_NL1*100),2)
NL1_time = round(NL_results1.cumulative_total_time/60,2)
NL1_fval = round(float(NL_results1.objective),2)
NL1_covariance_matrix = copy.deepcopy((NL_results1.parameter_covariances)/NL_results1.problem.N)
NL1_parameter_estimates  = copy.deepcopy(NL_results1.parameters)

# NL2 (Method = 2s; W Clustered = 0 ; Centered Moments = 0 )
NL2_price_coef = round(float(NL_results2.beta[0]),4)
NL2_price_se =  round(float(NL_results2.beta_se[0]),4)
NL2_nesting_coef = round(float(NL_results2.rho),4)
NL2_nesting_se = round(float(NL_results2.rho_se),4)
NL2_elast_own = round(float(Median_Own_Price_Elasticity_NL2),2)
NL2_elast_mkt =round(float(Median_Market_Price_Elasticity_NL2),2)
NL2_out_diversion = round(float(Median_Outside_Diversion_NL2*100),2)
NL2_time = round(NL_results2.cumulative_total_time/60,2)
NL2_fval = round(float(NL_results2.objective),2)
NL2_covariance_matrix = copy.deepcopy((NL_results2.parameter_covariances)/NL_results2.problem.N)
NL2_parameter_estimates  = copy.deepcopy(NL_results2.parameters)

# NL3 (Method = 2s; W Clustered = 1 ; Centered Moments = 0 )
NL3_price_coef = round(float(NL_results3.beta[0]),4)
NL3_price_se =  round(float(NL_results3.beta_se[0]),4)
NL3_nesting_coef = round(float(NL_results3.rho),4)
NL3_nesting_se = round(float(NL_results3.rho_se),4)
NL3_elast_own = round(float(Median_Own_Price_Elasticity_NL3),2)
NL3_elast_mkt =round(float(Median_Market_Price_Elasticity_NL3),2)
NL3_out_diversion = round(float(Median_Outside_Diversion_NL3*100),2)
NL3_time = round(NL_results3.cumulative_total_time/60,2)
NL3_fval = round(float(NL_results3.objective),2)
NL3_covariance_matrix = copy.deepcopy((NL_results3.parameter_covariances)/NL_results3.problem.N)
NL3_parameter_estimates  = copy.deepcopy(NL_results3.parameters)

# NL4 (Method = 2s; W Clustered = 1 ; Centered Moments = 1 )
NL4_price_coef = round(float(NL_results4.beta[0]),4)
NL4_price_se =  round(float(NL_results4.beta_se[0]),4)
NL4_nesting_coef = round(float(NL_results4.rho),4)
NL4_nesting_se = round(float(NL_results4.rho_se),4)
NL4_elast_own = round(float(Median_Own_Price_Elasticity_NL4),2)
NL4_elast_mkt =round(float(Median_Market_Price_Elasticity_NL4),2)
NL4_out_diversion = round(float(Median_Outside_Diversion_NL4*100),2)
NL4_time = round(NL_results4.cumulative_total_time/60,2)
NL4_fval = round(float(NL_results4.objective),2)
NL4_covariance_matrix = copy.deepcopy((NL_results4.parameter_covariances)/NL_results4.problem.N)
NL4_parameter_estimates  = copy.deepcopy(NL_results4.parameters)

# NL5 (Method = 2s; W Clustered = 0 ; Centered Moments = 1 )
NL5_price_coef = round(float(NL_results5.beta[0]),4)
NL5_price_se =  round(float(NL_results5.beta_se[0]),4)
NL5_nesting_coef = round(float(NL_results5.rho),4)
NL5_nesting_se = round(float(NL_results5.rho_se),4)
NL5_elast_own = round(float(Median_Own_Price_Elasticity_NL5),2)
NL5_elast_mkt =round(float(Median_Market_Price_Elasticity_NL5),2)
NL5_out_diversion = round(float(Median_Outside_Diversion_NL5*100),2)
NL5_time = round(NL_results5.cumulative_total_time/60,2)
NL5_fval = round(float(NL_results5.objective),2)
NL5_covariance_matrix = copy.deepcopy((NL_results5.parameter_covariances)/NL_results5.problem.N)
NL5_parameter_estimates  = copy.deepcopy(NL_results5.parameters)

In [86]:
# Output to CSV
path_output = ('XXX')
temp_covariance_1 = pd.DataFrame(NL1_covariance_matrix)
temp_covariance_1.to_csv(path_output + 'Output\\covariance_matrix_NL_1.csv',index= False)
temp_covariance_2 = pd.DataFrame(NL2_covariance_matrix)
temp_covariance_2.to_csv(path_output + 'Output\\covariance_matrix_NL_2.csv',index= False)
temp_covariance_3 = pd.DataFrame(NL3_covariance_matrix)
temp_covariance_3.to_csv(path_output + 'Output\\covariance_matrix_NL_3.csv',index= False)
temp_covariance_4 = pd.DataFrame(NL4_covariance_matrix)
temp_covariance_4.to_csv(path_output + 'Output\\covariance_matrix_NL_4.csv',index= False)
temp_covariance_5 = pd.DataFrame(NL5_covariance_matrix)
temp_covariance_5.to_csv(path_output + 'Output\\covariance_matrix_NL_5.csv',index= False)

temp_parameter_1 = pd.DataFrame(NL1_parameter_estimates)
temp_parameter_1.to_csv(path + 'output\\parameter_estimates_NL_1.csv',index= False)
temp_parameter_2 = pd.DataFrame(NL2_parameter_estimates)
temp_parameter_2.to_csv(path + 'output\\parameter_estimates_NL_2.csv',index= False)
temp_parameter_3 = pd.DataFrame(NL3_parameter_estimates)
temp_parameter_3.to_csv(path + 'output\\parameter_estimates_NL_3.csv',index= False)
temp_parameter_4 = pd.DataFrame(NL4_parameter_estimates)
temp_parameter_4.to_csv(path + 'output\\parameter_estimates_NL_4.csv',index= False)
temp_parameter_5 = pd.DataFrame(NL5_parameter_estimates)
temp_parameter_5.to_csv(path + 'output\\parameter_estimates_NL_5.csv',index= False)

In [87]:
table_estimates_NL =   [["Variable","Parameter","ECMA","Replication","(1)","(2)","(3)","(4)","(5)"],
                    ["Price","$alpha$", ecma_price_coef, matlab_price_coef, NL1_price_coef, NL2_price_coef, NL3_price_coef, NL4_price_coef, NL5_price_coef],
                    ["Price SE","", '('+str(ecma_price_se)+')', '('+str(matlab_price_se)+')', '('+str(NL1_price_se)+')', '('+str(NL2_price_se)+')', '('+str(NL3_price_se)+')', '('+str(NL4_price_se)+')', '('+str(NL5_price_se)+')'],
                    ["Nesting Parameter","$rho$", ecma_nesting_coef, matlab_nesting_coef, NL1_nesting_coef, NL2_nesting_coef, NL3_nesting_coef, NL4_nesting_coef, NL5_nesting_coef],
                    ["Nesting SE","", '('+str(ecma_nesting_se)+')', '('+str(matlab_nesting_se)+')',  '('+str(NL1_nesting_se)+')',  '('+str(NL2_nesting_se)+')',  '('+str(NL3_nesting_se)+')',  '('+str(NL4_nesting_se)+')',  '('+str(NL5_nesting_se)+')'],
                    ["Other Statistics"],
                    ["Median Own Price Elasticity","",ecma_elast_own, matlab_elast_own, NL1_elast_own, NL2_elast_own, NL3_elast_own, NL4_elast_own, NL5_elast_own],
                    ["Median Market Price Elasticity","",ecma_elast_mkt,matlab_elast_mkt, NL1_elast_mkt, NL2_elast_mkt, NL3_elast_mkt, NL4_elast_mkt, NL5_elast_mkt],
                    ["Median Outside Diversion (%)","",ecma_out_diversion,matlab_out_diversion, NL1_out_diversion, NL2_out_diversion, NL3_out_diversion, NL4_out_diversion, NL5_out_diversion],
                    ["Fval","","","",NL1_fval,NL2_fval,NL3_fval,NL4_fval,NL5_fval],
                    ["Observations","",Obs,Obs,Obs,Obs,Obs,Obs,Obs],
                    ["Method","","2SLS","2SLS","GMM1s","GMM2s","GMM2s","GMM2s","GMM2s"],
                    ["Clustered W","","-","-","-","No","Yes","Yes","No"],
                    ["Centered Moments","","-","-","-","No","No","Yes","Yes"],
                    ["Software","","Matlab","Matlab","PyBLP","PyBLP","PyBLP","PyBLP","PyBLP"],
                    ["Time (min)","","-","-",NL1_time,NL2_time,NL3_time,NL4_time,NL5_time]] 
table_NL = tabulate.tabulate(table_estimates_NL,headers="firstrow")
df_NL = pd.DataFrame(table_estimates_NL)
df_NL.to_csv(path_output + 'Output\\table_NL.csv',index=False)
df_NL

,0,1,2,3,4,5,6,7,8
0,Variable,Parameter,ECMA,Replication,(1),(2),(3),(4),(5)
1,Price,$alpha$,-0.1312,-0.1243,-0.1227,-0.1327,-0.0558,0.0006,-0.1335
2,Price SE,,(0.0884),(0.0903),(0.0887),(0.1099),(0.0167),(0.0118),(0.1101)
3,Nesting Parameter,$rho$,0.6299,0.6349,0.638,0.5634,0.7406,0.8288,0.5569
4,Nesting SE,,(0.0941),(0.0909),(0.0902),(0.1083),(0.0499),(0.039),(0.1101)
5,Other Statistics,None,None,None,None,None,None,None,None
6,Median Own Price Elasticity,,-3.81,-3.66,-3.64,-3.27,-2.31,0.04,-3.24
7,Median Market Price Elasticity,,-1.1,-1.04,-0.98,-1.06,-0.44,0.01,-1.06
8,Median Outside Diversion (%),,29.8,29.49,27.66,33.32,19.86,13.12,33.81
9,Fval,,,,1854.61,7544.62,17.89,33.44,8198.05


### Analysis of Random Coefficients Logit (RCNL-1)

#### 1. Estimation of the RCNL-1

In [16]:
# RCNL1 (Method = 2s; Optimization = l-bfgs-b; W Clustered = 1 ; Centered Moments = 1 )
# Starting values
rho_start = 0.5
pi_start = [0.035, 0.0002, 0.004]

# Formulation for the linear and nonlinear parts of the model. 
def solve_rcnl1(df_prod,df_agent):
    df_prod['nesting_ids'] = 1
    #product_formulations = (X1_formulation, X2_formulation)
    product_formulations = (pyblp.Formulation('0 + prices', absorb='C(product_ids)+C(date)'),
                            pyblp.Formulation('1 + prices + calories_144'))
    agent_formulation = pyblp.Formulation('0 + income')

    problem = pyblp.Problem(product_formulations, df_prod,agent_formulation,df_agent)
    initial_sigma = np.zeros((3,3))
    initial_pi = np.c_[[pi_start[0], pi_start[1], pi_start[2]]]
    tighter_opt = pyblp.Optimization('l-bfgs-b', {'ftol': 0, 'gtol': 1e-4})
    results = problem.solve(initial_sigma,initial_pi,rho=rho_start,fp_type='safe_linear',optimization=tighter_opt,scale_objective=False,se_type='clustered',W_type='clustered',center_moments=True)
    return results

# Solve the problem
data1 = prep_data()
data_product_iv = prod_w_iv(data1)
agent_data = prep_agent_data(data_product_iv)
RCNL_results1 = solve_rcnl1(data_product_iv,agent_data)

# Compute economic stats
elasticities_RCNL1 = RCNL_results1.compute_elasticities()
diversions_RCNL1 = RCNL_results1.compute_diversion_ratios()
Own_Price_Elasticity_RCNL1 = RCNL_results1.extract_diagonals(elasticities_RCNL1)
Median_Own_Price_Elasticity_RCNL1 = np.quantile(Own_Price_Elasticity_RCNL1, 0.5)
Outside_Diversion_RCNL1 = RCNL_results1.extract_diagonals(diversions_RCNL1)
Median_Outside_Diversion_RCNL1 = np.quantile(Outside_Diversion_RCNL1, 0.5)
aggregates_RCNL1 = RCNL_results1.compute_aggregate_elasticities(factor=0.1)
Median_agg_demand_elas_RCNL1 = np.quantile(aggregates_RCNL1, 0.5)
#Vector with cross-elasticity to outside good
ielas_RCNL1 = Own_Price_Elasticity_RCNL1 * Outside_Diversion_RCNL1
Median_Market_Price_Elasticity_RCNL1 = np.quantile(ielas_RCNL1, 0.5)
J_stat_RCNL1 = RCNL_results1.run_hansen_test()

====After excluding year after merge approved, sample size is 94656 ====
==== Number of rows (years(7)*mkts(39)*individuals(500)) should be 136500-->136500 ====


In [17]:
# RCNL2 (Method = 2s; Optimization = trust-constr; W Clustered = 1 ; Centered Moments = 1 )
# Starting values
rho_start = 0.5
pi_start = [0.035, 0.0002, 0.004]

# Formulation for the linear and nonlinear parts of the model. 
def solve_rcnl1(df_prod,df_agent):
    df_prod['nesting_ids'] = 1
    #product_formulations = (X1_formulation, X2_formulation)
    product_formulations = (pyblp.Formulation('0 + prices', absorb='C(product_ids)+C(date)'),
                            pyblp.Formulation('1 + prices + calories_144'))
    agent_formulation = pyblp.Formulation('0 + income')

    problem = pyblp.Problem(product_formulations, df_prod,agent_formulation,df_agent)
    initial_sigma = np.zeros((3,3))
    initial_pi = np.c_[[pi_start[0], pi_start[1], pi_start[2]]]
    tighter_opt = pyblp.Optimization('trust-constr', {'gtol': 1e-4, 'xtol': 1e-4})
    results = problem.solve(initial_sigma,initial_pi,rho=rho_start,fp_type='safe_linear',optimization=tighter_opt,scale_objective=False,se_type='clustered',W_type='clustered',center_moments=True)
    return results

# Solve the problem
data1 = prep_data()
data_product_iv = prod_w_iv(data1)
agent_data = prep_agent_data(data_product_iv)
RCNL_results2 = solve_rcnl1(data_product_iv,agent_data)

# Compute economic stats
elasticities_RCNL2 = RCNL_results2.compute_elasticities()
diversions_RCNL2 = RCNL_results2.compute_diversion_ratios()
Own_Price_Elasticity_RCNL2 = RCNL_results2.extract_diagonals(elasticities_RCNL2)
Median_Own_Price_Elasticity_RCNL2 = np.quantile(Own_Price_Elasticity_RCNL2, 0.5)
Outside_Diversion_RCNL2 = RCNL_results2.extract_diagonals(diversions_RCNL2)
Median_Outside_Diversion_RCNL2 = np.quantile(Outside_Diversion_RCNL2, 0.5)
aggregates_RCNL2 = RCNL_results2.compute_aggregate_elasticities(factor=0.1)
Median_agg_demand_elas_RCNL2 = np.quantile(aggregates_RCNL2, 0.5)
#Vector with cross-elasticity to outside good
ielas_RCNL2 = Own_Price_Elasticity_RCNL2 * Outside_Diversion_RCNL2
Median_Market_Price_Elasticity_RCNL2 = np.quantile(ielas_RCNL2, 0.5)
J_stat_RCNL2 = RCNL_results2.run_hansen_test()

====After excluding year after merge approved, sample size is 94656 ====
==== Number of rows (years(7)*mkts(39)*individuals(500)) should be 136500-->136500 ====


In [18]:
# RCNL3 (Method = 2s; Optimization = trust-constr; W Clustered = 1 ; Centered Moments = 0 )
# Starting values
rho_start = 0.5
pi_start = [0.035, 0.0002, 0.004]

# Formulation for the linear and nonlinear parts of the model. 
def solve_rcnl1(df_prod,df_agent):
    df_prod['nesting_ids'] = 1
    #product_formulations = (X1_formulation, X2_formulation)
    product_formulations = (pyblp.Formulation('0 + prices', absorb='C(product_ids)+C(date)'),
                            pyblp.Formulation('1 + prices + calories_144'))
    agent_formulation = pyblp.Formulation('0 + income')

    problem = pyblp.Problem(product_formulations, df_prod,agent_formulation,df_agent)
    initial_sigma = np.zeros((3,3))
    initial_pi = np.c_[[pi_start[0], pi_start[1], pi_start[2]]]
    tighter_opt = pyblp.Optimization('trust-constr', {'gtol': 1e-4, 'xtol': 1e-4})
    results = problem.solve(initial_sigma,initial_pi,rho=rho_start,fp_type='safe_linear',optimization=tighter_opt,scale_objective=False,se_type='clustered',W_type='clustered',center_moments=False)
    return results

# Solve the problem
data1 = prep_data()
data_product_iv = prod_w_iv(data1)
agent_data = prep_agent_data(data_product_iv)
RCNL_results3 = solve_rcnl1(data_product_iv,agent_data)

# Compute economic stats
elasticities_RCNL3 = RCNL_results3.compute_elasticities()
diversions_RCNL3 = RCNL_results3.compute_diversion_ratios()
Own_Price_Elasticity_RCNL3 = RCNL_results3.extract_diagonals(elasticities_RCNL3)
Median_Own_Price_Elasticity_RCNL3 = np.quantile(Own_Price_Elasticity_RCNL3, 0.5)
Outside_Diversion_RCNL3 = RCNL_results3.extract_diagonals(diversions_RCNL3)
Median_Outside_Diversion_RCNL3 = np.quantile(Outside_Diversion_RCNL3, 0.5)
aggregates_RCNL3 = RCNL_results3.compute_aggregate_elasticities(factor=0.1)
Median_agg_demand_elas_RCNL3 = np.quantile(aggregates_RCNL3, 0.5)
#Vector with cross-elasticity to outside good
ielas_RCNL3 = Own_Price_Elasticity_RCNL3 * Outside_Diversion_RCNL3
Median_Market_Price_Elasticity_RCNL3 = np.quantile(ielas_RCNL3, 0.5)
J_stat_RCNL3 = RCNL_results3.run_hansen_test()

====After excluding year after merge approved, sample size is 94656 ====
==== Number of rows (years(7)*mkts(39)*individuals(500)) should be 136500-->136500 ====


In [19]:
# RCNL4 (Method = 2s; Optimization = nelder-mead; W Clustered = 1 ; Centered Moments = 0 )
# Starting values
rho_start = 0.5
pi_start = [0.035, 0.0002, 0.004]

# Formulation for the linear and nonlinear parts of the model. 
def solve_rcnl1(df_prod,df_agent):
    df_prod['nesting_ids'] = 1
    #product_formulations = (X1_formulation, X2_formulation)
    product_formulations = (pyblp.Formulation('0 + prices', absorb='C(product_ids)+C(date)'),
                            pyblp.Formulation('1 + prices + calories_144'))
    agent_formulation = pyblp.Formulation('0 + income')

    problem = pyblp.Problem(product_formulations, df_prod,agent_formulation,df_agent)
    initial_sigma = np.zeros((3,3))
    initial_pi = np.c_[[pi_start[0], pi_start[1], pi_start[2]]]
    tighter_opt = pyblp.Optimization('nelder-mead',{'xatol': 1e-4, 'fatol': 1e-4},compute_gradient=False)
    results = problem.solve(initial_sigma,initial_pi,rho=rho_start,fp_type='safe_linear',optimization=tighter_opt,scale_objective=False,se_type='clustered',W_type='clustered',center_moments=False)
    return results

# Solve the problem
data1 = prep_data()
data_product_iv = prod_w_iv(data1)
agent_data = prep_agent_data(data_product_iv)
RCNL_results4 = solve_rcnl1(data_product_iv,agent_data)

# Compute economic stats
elasticities_RCNL4 = RCNL_results4.compute_elasticities()
diversions_RCNL4 = RCNL_results4.compute_diversion_ratios()
Own_Price_Elasticity_RCNL4 = RCNL_results4.extract_diagonals(elasticities_RCNL4)
Median_Own_Price_Elasticity_RCNL4 = np.quantile(Own_Price_Elasticity_RCNL4, 0.5)
Outside_Diversion_RCNL4 = RCNL_results4.extract_diagonals(diversions_RCNL4)
Median_Outside_Diversion_RCNL4 = np.quantile(Outside_Diversion_RCNL4, 0.5)
aggregates_RCNL4 = RCNL_results4.compute_aggregate_elasticities(factor=0.1)
Median_agg_demand_elas_RCNL4 = np.quantile(aggregates_RCNL4, 0.5)
#Vector with cross-elasticity to outside good
ielas_RCNL4 = Own_Price_Elasticity_RCNL4 * Outside_Diversion_RCNL4
Median_Market_Price_Elasticity_RCNL4 = np.quantile(ielas_RCNL4, 0.5)
J_stat_RCNL4 = RCNL_results4.run_hansen_test()

====After excluding year after merge approved, sample size is 94656 ====
==== Number of rows (years(7)*mkts(39)*individuals(500)) should be 136500-->136500 ====


In [20]:
# RCNL5 (Method = 2s; Optimization = nelder-mead; W Clustered = 1 ; Centered Moments = 1 )
# Starting values
rho_start = 0.5
pi_start = [0.035, 0.0002, 0.004]

# Formulation for the linear and nonlinear parts of the model. 
def solve_rcnl1(df_prod,df_agent):
    df_prod['nesting_ids'] = 1
    #product_formulations = (X1_formulation, X2_formulation)
    product_formulations = (pyblp.Formulation('0 + prices', absorb='C(product_ids)+C(date)'),
                            pyblp.Formulation('1 + prices + calories_144'))
    agent_formulation = pyblp.Formulation('0 + income')

    problem = pyblp.Problem(product_formulations, df_prod,agent_formulation,df_agent)
    initial_sigma = np.zeros((3,3))
    initial_pi = np.c_[[pi_start[0], pi_start[1], pi_start[2]]]
    tighter_opt = pyblp.Optimization('nelder-mead',{'xatol': 1e-4, 'fatol': 1e-4},compute_gradient=False)
    results = problem.solve(initial_sigma,initial_pi,rho=rho_start,fp_type='safe_linear',optimization=tighter_opt,scale_objective=False,se_type='clustered',W_type='clustered',center_moments=True)
    return results

# Solve the problem
data1 = prep_data()
data_product_iv = prod_w_iv(data1)
agent_data = prep_agent_data(data_product_iv)
RCNL_results5 = solve_rcnl1(data_product_iv,agent_data)

# Compute economic stats
elasticities_RCNL5 = RCNL_results5.compute_elasticities()
diversions_RCNL5 = RCNL_results5.compute_diversion_ratios()
Own_Price_Elasticity_RCNL5 = RCNL_results5.extract_diagonals(elasticities_RCNL5)
Median_Own_Price_Elasticity_RCNL5 = np.quantile(Own_Price_Elasticity_RCNL5, 0.5)
Outside_Diversion_RCNL5 = RCNL_results5.extract_diagonals(diversions_RCNL5)
Median_Outside_Diversion_RCNL5 = np.quantile(Outside_Diversion_RCNL5, 0.5)
aggregates_RCNL5 = RCNL_results5.compute_aggregate_elasticities(factor=0.1)
Median_agg_demand_elas_RCNL5 = np.quantile(aggregates_RCNL5, 0.5)
#Vector with cross-elasticity to outside good
ielas_RCNL5 = Own_Price_Elasticity_RCNL5 * Outside_Diversion_RCNL5
Median_Market_Price_Elasticity_RCNL5 = np.quantile(ielas_RCNL5, 0.5)
J_stat_RCNL5 = RCNL_results5.run_hansen_test()

====After excluding year after merge approved, sample size is 94656 ====
==== Number of rows (years(7)*mkts(39)*individuals(500)) should be 136500-->136500 ====


In [21]:
# RCNL6 (Method = 2s; Optimization = l-bfgs-b; W Clustered = 1 ; Centered Moments = 0 )
# Starting values
rho_start = 0.5
pi_start = [0.035, 0.0002, 0.004]

# Formulation for the linear and nonlinear parts of the model. 
def solve_rcnl1(df_prod,df_agent):
    df_prod['nesting_ids'] = 1
    #product_formulations = (X1_formulation, X2_formulation)
    product_formulations = (pyblp.Formulation('0 + prices', absorb='C(product_ids)+C(date)'),
                            pyblp.Formulation('1 + prices + calories_144'))
    agent_formulation = pyblp.Formulation('0 + income')

    problem = pyblp.Problem(product_formulations, df_prod,agent_formulation,df_agent)
    initial_sigma = np.zeros((3,3))
    initial_pi = np.c_[[pi_start[0], pi_start[1], pi_start[2]]]
    tighter_opt = pyblp.Optimization('l-bfgs-b', {'ftol': 0, 'gtol': 1e-4})
    results = problem.solve(initial_sigma,initial_pi,rho=rho_start,fp_type='safe_linear',optimization=tighter_opt,scale_objective=False,se_type='clustered',W_type='clustered',center_moments=False)
    return results

# Solve the problem
data1 = prep_data()
data_product_iv = prod_w_iv(data1)
agent_data = prep_agent_data(data_product_iv)
RCNL_results6 = solve_rcnl1(data_product_iv,agent_data)

# Compute economic stats
elasticities_RCNL6 = RCNL_results6.compute_elasticities()
diversions_RCNL6 = RCNL_results6.compute_diversion_ratios()
Own_Price_Elasticity_RCNL6 = RCNL_results6.extract_diagonals(elasticities_RCNL6)
Median_Own_Price_Elasticity_RCNL6 = np.quantile(Own_Price_Elasticity_RCNL6, 0.5)
Outside_Diversion_RCNL6 = RCNL_results6.extract_diagonals(diversions_RCNL6)
Median_Outside_Diversion_RCNL6 = np.quantile(Outside_Diversion_RCNL6, 0.5)
aggregates_RCNL6 = RCNL_results6.compute_aggregate_elasticities(factor=0.1)
Median_agg_demand_elas_RCNL6 = np.quantile(aggregates_RCNL6, 0.5)
#Vector with cross-elasticity to outside good
ielas_RCNL6 = Own_Price_Elasticity_RCNL6 * Outside_Diversion_RCNL6
Median_Market_Price_Elasticity_RCNL6 = np.quantile(ielas_RCNL6, 0.5)
J_stat_RCNL6 = RCNL_results6.run_hansen_test()

====After excluding year after merge approved, sample size is 94656 ====
==== Number of rows (years(7)*mkts(39)*individuals(500)) should be 136500-->136500 ====


In [39]:
# RCNL7 (Method = 2s; Optimization = l-bfgs-b; W Clustered = 0 ; Centered Moments = 0 )
# Starting values
rho_start = 0.5
pi_start = [0.035, 0.0002, 0.004]

# Formulation for the linear and nonlinear parts of the model. 
def solve_rcnl1(df_prod,df_agent):
    df_prod['nesting_ids'] = 1
    #product_formulations = (X1_formulation, X2_formulation)
    product_formulations = (pyblp.Formulation('0 + prices', absorb='C(product_ids)+C(date)'),
                            pyblp.Formulation('1 + prices + calories_144'))
    agent_formulation = pyblp.Formulation('0 + income')

    problem = pyblp.Problem(product_formulations, df_prod,agent_formulation,df_agent)
    initial_sigma = np.zeros((3,3))
    initial_pi = np.c_[[pi_start[0], pi_start[1], pi_start[2]]]
    tighter_opt = pyblp.Optimization('l-bfgs-b', {'ftol': 0, 'gtol': 1e-4})
    results = problem.solve(initial_sigma,initial_pi,rho=rho_start,fp_type='safe_linear',optimization=tighter_opt,scale_objective=False,se_type='clustered',W_type='robust',center_moments=False)
    return results

# Solve the problem
data1 = prep_data()
data_product_iv = prod_w_iv(data1)
agent_data = prep_agent_data(data_product_iv)
RCNL_results7 = solve_rcnl1(data_product_iv,agent_data)

# Compute economic stats
elasticities_RCNL7 = RCNL_results7.compute_elasticities()
diversions_RCNL7 = RCNL_results7.compute_diversion_ratios()
Own_Price_Elasticity_RCNL7 = RCNL_results7.extract_diagonals(elasticities_RCNL7)
Median_Own_Price_Elasticity_RCNL7 = np.quantile(Own_Price_Elasticity_RCNL7, 0.5)
Outside_Diversion_RCNL7 = RCNL_results7.extract_diagonals(diversions_RCNL7)
Median_Outside_Diversion_RCNL7 = np.quantile(Outside_Diversion_RCNL7, 0.5)
aggregates_RCNL7 = RCNL_results7.compute_aggregate_elasticities(factor=0.1)
Median_agg_demand_elas_RCNL7 = np.quantile(aggregates_RCNL7, 0.5)
#Vector with cross-elasticity to outside good
ielas_RCNL7 = Own_Price_Elasticity_RCNL7 * Outside_Diversion_RCNL7
Median_Market_Price_Elasticity_RCNL7 = np.quantile(ielas_RCNL7, 0.5)
J_stat_RCNL7 = RCNL_results7.run_hansen_test()

====After excluding year after merge approved, sample size is 94656 ====
==== Number of rows (years(7)*mkts(39)*individuals(500)) should be 136500-->136500 ====


#### 2. Store the output

In [74]:
# Demand estimates in ECMA paper and Matlab
Obs = data1.shape[0]
ecma_price_coef = -0.0887
ecma_price_se =  0.0141
ecma_nesting_coef = 0.8299
ecma_nesting_se =  0.0402
ecma_income_price = 0.0007
ecma_income_price_se = 0.0002
ecma_income_constant = 0.0143
ecma_income_constant_se = 0.0051
ecma_income_calories = 0.0043
ecma_income_calories_se = 0.0016
ecma_elast_own = -4.74
ecma_elast_mkt = -0.60
ecma_out_diversion = round(0.1296*100,2)
ecma_fval = round(13.94/Obs,4)
ecma_Jstat = 13.94

matlab_price_coef = -0.0841
matlab_price_se =  0.0137
matlab_nesting_coef = 0.8359
matlab_nesting_se =  0.0389
matlab_income_price = 0.0006
matlab_income_price_se = 0.0002
matlab_income_constant = 0.0147
matlab_income_constant_se = 0.0049
matlab_income_calories = 0.0040
matlab_income_calories_se = 0.0015
matlab_elast_own = round(-4.7180,2)
matlab_elast_mkt = round(-0.5761,2)
matlab_out_diversion = round(0.1253*100,2)
matlab_time = round(31.9318*60,2)
matlab_fval = round(float(13.4797/Obs),4)
matlab_Jstat = round(float(13.4797),2)

# RCNL1 (Method = 2s; Optimization = l-bfgs-b; W Clustered = 1 ; Centered Moments = 1 )
RCNL1_price_coef = round(float(RCNL_results1.beta[0]),4)
RCNL1_price_se =  round(float(RCNL_results1.beta_se[0]),4)
RCNL1_nesting_coef = round(float(RCNL_results1.rho),4)
RCNL1_nesting_se = round(float(RCNL_results1.rho_se),4)
RCNL1_income_price = round(float(RCNL_results1.pi[1]),4)
RCNL1_income_price_se = round(float(RCNL_results1.pi_se[1]),4)
RCNL1_income_constant = round(float(RCNL_results1.pi[0]),4)
RCNL1_income_constant_se = round(float(RCNL_results1.pi_se[0]),4)
RCNL1_income_calories = round(float(RCNL_results1.pi[2]),4)
RCNL1_income_calories_se = round(float(RCNL_results1.pi_se[2]),4)
RCNL1_elast_own = round(float(Median_Own_Price_Elasticity_RCNL1),2)
RCNL1_elast_mkt =round(float(Median_Market_Price_Elasticity_RCNL1),2)
RCNL1_out_diversion = round(float(Median_Outside_Diversion_RCNL1*100),2)
RCNL1_time = round(RCNL_results1.cumulative_total_time/60,2)
RCNL1_fval = round(float(RCNL_results1.objective),2)
RCNL1_Jstat = round(J_stat_RCNL1,2)
RCNL1_covariance_matrix = copy.deepcopy((RCNL_results1.parameter_covariances)/RCNL_results1.problem.N)
RCNL1_parameter_estimates  = copy.deepcopy(RCNL_results1.parameters)

# RCNL2 (Method = 2s; Optimization = trust-constr; W Clustered = 1 ; Centered Moments = 1 )
RCNL2_price_coef = round(float(RCNL_results2.beta[0]),4)
RCNL2_price_se =  round(float(RCNL_results2.beta_se[0]),4)
RCNL2_nesting_coef = round(float(RCNL_results2.rho),4)
RCNL2_nesting_se = round(float(RCNL_results2.rho_se),4)
RCNL2_income_price = round(float(RCNL_results2.pi[1]),4)
RCNL2_income_price_se = round(float(RCNL_results2.pi_se[1]),4)
RCNL2_income_constant = round(float(RCNL_results2.pi[0]),4)
RCNL2_income_constant_se = round(float(RCNL_results2.pi_se[0]),4)
RCNL2_income_calories = round(float(RCNL_results2.pi[2]),4)
RCNL2_income_calories_se = round(float(RCNL_results2.pi_se[2]),4)
RCNL2_elast_own = round(float(Median_Own_Price_Elasticity_RCNL2),2)
RCNL2_elast_mkt =round(float(Median_Market_Price_Elasticity_RCNL2),2)
RCNL2_out_diversion = round(float(Median_Outside_Diversion_RCNL2*100),2)
RCNL2_time = round(RCNL_results2.cumulative_total_time/60,2)
RCNL2_fval = round(float(RCNL_results2.objective),2)
RCNL2_Jstat = round(J_stat_RCNL2,2)
RCNL2_covariance_matrix = copy.deepcopy((RCNL_results2.parameter_covariances)/RCNL_results2.problem.N)
RCNL2_parameter_estimates  = copy.deepcopy(RCNL_results2.parameters)

# RCNL3 (Method = 2s; Optimization = trust-constr; W Clustered = 1 ; Centered Moments = 0 )
RCNL3_price_coef = round(float(RCNL_results3.beta[0]),4)
RCNL3_price_se =  round(float(RCNL_results3.beta_se[0]),4)
RCNL3_nesting_coef = round(float(RCNL_results3.rho),4)
RCNL3_nesting_se = round(float(RCNL_results3.rho_se),4)
RCNL3_income_price = round(float(RCNL_results3.pi[1]),4)
RCNL3_income_price_se = round(float(RCNL_results3.pi_se[1]),4)
RCNL3_income_constant = round(float(RCNL_results3.pi[0]),4)
RCNL3_income_constant_se = round(float(RCNL_results3.pi_se[0]),4)
RCNL3_income_calories = round(float(RCNL_results3.pi[2]),4)
RCNL3_income_calories_se = round(float(RCNL_results3.pi_se[2]),4)
RCNL3_elast_own = round(float(Median_Own_Price_Elasticity_RCNL3),2)
RCNL3_elast_mkt =round(float(Median_Market_Price_Elasticity_RCNL3),2)
RCNL3_out_diversion = round(float(Median_Outside_Diversion_RCNL3*100),2)
RCNL3_time = round(RCNL_results3.cumulative_total_time/60,2)
RCNL3_fval = round(float(RCNL_results3.objective),2)
RCNL3_Jstat = round(J_stat_RCNL3,2)
RCNL3_covariance_matrix = copy.deepcopy((RCNL_results3.parameter_covariances)/RCNL_results3.problem.N)
RCNL3_parameter_estimates  = copy.deepcopy(RCNL_results3.parameters)

# RCNL4 (Method = 2s; Optimization = nelder-mead; W Clustered = 1 ; Centered Moments = 0 )
RCNL4_price_coef = round(float(RCNL_results4.beta[0]),4)
RCNL4_price_se =  round(float(RCNL_results4.beta_se[0]),4)
RCNL4_nesting_coef = round(float(RCNL_results4.rho),4)
RCNL4_nesting_se = round(float(RCNL_results4.rho_se),4)
RCNL4_income_price = round(float(RCNL_results4.pi[1]),4)
RCNL4_income_price_se = round(float(RCNL_results4.pi_se[1]),4)
RCNL4_income_constant = round(float(RCNL_results4.pi[0]),4)
RCNL4_income_constant_se = round(float(RCNL_results4.pi_se[0]),4)
RCNL4_income_calories = round(float(RCNL_results4.pi[2]),4)
RCNL4_income_calories_se = round(float(RCNL_results4.pi_se[2]),4)
RCNL4_elast_own = round(float(Median_Own_Price_Elasticity_RCNL4),2)
RCNL4_elast_mkt =round(float(Median_Market_Price_Elasticity_RCNL4),2)
RCNL4_out_diversion = round(float(Median_Outside_Diversion_RCNL4*100),2)
RCNL4_time = round(RCNL_results4.cumulative_total_time/60,2)
RCNL4_fval = round(float(RCNL_results4.objective),2)
RCNL4_Jstat = round(J_stat_RCNL4,2)
RCNL4_covariance_matrix = copy.deepcopy((RCNL_results4.parameter_covariances)/RCNL_results4.problem.N)
RCNL4_parameter_estimates  = copy.deepcopy(RCNL_results4.parameters)

# RCNL5 (Method = 2s; Optimization = nelder-mead; W Clustered = 1 ; Centered Moments = 1 )
RCNL5_price_coef = round(float(RCNL_results5.beta[0]),4)
RCNL5_price_se =  round(float(RCNL_results5.beta_se[0]),4)
RCNL5_nesting_coef = round(float(RCNL_results5.rho),4)
RCNL5_nesting_se = round(float(RCNL_results5.rho_se),4)
RCNL5_income_price = round(float(RCNL_results5.pi[1]),4)
RCNL5_income_price_se = round(float(RCNL_results5.pi_se[1]),4)
RCNL5_income_constant = round(float(RCNL_results5.pi[0]),4)
RCNL5_income_constant_se = round(float(RCNL_results5.pi_se[0]),4)
RCNL5_income_calories = round(float(RCNL_results5.pi[2]),4)
RCNL5_income_calories_se = round(float(RCNL_results5.pi_se[2]),4)
RCNL5_elast_own = round(float(Median_Own_Price_Elasticity_RCNL5),2)
RCNL5_elast_mkt =round(float(Median_Market_Price_Elasticity_RCNL5),2)
RCNL5_out_diversion = round(float(Median_Outside_Diversion_RCNL5*100),2)
RCNL5_time = round(RCNL_results5.cumulative_total_time/60,2)
RCNL5_fval = round(float(RCNL_results5.objective),2)
RCNL5_Jstat = round(J_stat_RCNL5,2)
RCNL5_covariance_matrix = copy.deepcopy((RCNL_results5.parameter_covariances)/RCNL_results5.problem.N)
RCNL5_parameter_estimates  = copy.deepcopy(RCNL_results5.parameters)

# RCNL6 (Method = 2s; Optimization = l-bfgs-b; W Clustered = 1 ; Centered Moments = 0 )
RCNL6_price_coef = round(float(RCNL_results6.beta[0]),4)
RCNL6_price_se =  round(float(RCNL_results6.beta_se[0]),4)
RCNL6_nesting_coef = round(float(RCNL_results6.rho),4)
RCNL6_nesting_se = round(float(RCNL_results6.rho_se),4)
RCNL6_income_price = round(float(RCNL_results6.pi[1]),4)
RCNL6_income_price_se = round(float(RCNL_results6.pi_se[1]),4)
RCNL6_income_constant = round(float(RCNL_results6.pi[0]),4)
RCNL6_income_constant_se = round(float(RCNL_results6.pi_se[0]),4)
RCNL6_income_calories = round(float(RCNL_results6.pi[2]),4)
RCNL6_income_calories_se = round(float(RCNL_results6.pi_se[2]),4)
RCNL6_elast_own = round(float(Median_Own_Price_Elasticity_RCNL6),2)
RCNL6_elast_mkt =round(float(Median_Market_Price_Elasticity_RCNL6),2)
RCNL6_out_diversion = round(float(Median_Outside_Diversion_RCNL6*100),2)
RCNL6_time = round(RCNL_results6.cumulative_total_time/60,2)
RCNL6_fval = round(float(RCNL_results6.objective),2)
RCNL6_Jstat = round(J_stat_RCNL6,2)
RCNL6_covariance_matrix = copy.deepcopy((RCNL_results6.parameter_covariances)/RCNL_results6.problem.N)
RCNL6_parameter_estimates  = copy.deepcopy(RCNL_results6.parameters)

# RCNL7 (Method = 2s; Optimization = l-bfgs-b; W Clustered = 0 ; Centered Moments = 0 )
RCNL7_price_coef = round(float(RCNL_results7.beta[0]),4)
RCNL7_price_se =  round(float(RCNL_results7.beta_se[0]),4)
RCNL7_nesting_coef = round(float(RCNL_results7.rho),4)
RCNL7_nesting_se = round(float(RCNL_results7.rho_se),4)
RCNL7_income_price = round(float(RCNL_results7.pi[1]),4)
RCNL7_income_price_se = round(float(RCNL_results7.pi_se[1]),4)
RCNL7_income_constant = round(float(RCNL_results7.pi[0]),4)
RCNL7_income_constant_se = round(float(RCNL_results7.pi_se[0]),4)
RCNL7_income_calories = round(float(RCNL_results7.pi[2]),4)
RCNL7_income_calories_se = round(float(RCNL_results7.pi_se[2]),4)
RCNL7_elast_own = round(float(Median_Own_Price_Elasticity_RCNL7),2)
RCNL7_elast_mkt =round(float(Median_Market_Price_Elasticity_RCNL7),2)
RCNL7_out_diversion = round(float(Median_Outside_Diversion_RCNL7*100),2)
RCNL7_time = round(RCNL_results7.cumulative_total_time/60,2)
RCNL7_fval = round(float(RCNL_results7.objective),2)
RCNL7_Jstat = round(J_stat_RCNL7,2)
RCNL7_covariance_matrix = copy.deepcopy((RCNL_results7.parameter_covariances)/RCNL_results7.problem.N)
RCNL7_parameter_estimates  = copy.deepcopy(RCNL_results7.parameters)

In [81]:
# Output to CSV
path_output = ('XXX')
temp_covariance_1 = pd.DataFrame(RCNL1_covariance_matrix)
temp_covariance_1.to_csv(path_output + 'Output\\covariance_matrix_RCNL_1.csv',index= False)
temp_covariance_2 = pd.DataFrame(RCNL2_covariance_matrix)
temp_covariance_2.to_csv(path_output + 'Output\\covariance_matrix_RCNL_2.csv',index= False)
temp_covariance_3 = pd.DataFrame(RCNL3_covariance_matrix)
temp_covariance_3.to_csv(path_output + 'Output\\covariance_matrix_RCNL_3.csv',index= False)
temp_covariance_4 = pd.DataFrame(RCNL4_covariance_matrix)
temp_covariance_4.to_csv(path_output + 'Output\\covariance_matrix_RCNL_4.csv',index= False)
temp_covariance_5 = pd.DataFrame(RCNL5_covariance_matrix)
temp_covariance_5.to_csv(path_output + 'Output\\covariance_matrix_RCNL_5.csv',index= False)
temp_covariance_6 = pd.DataFrame(RCNL6_covariance_matrix)
temp_covariance_6.to_csv(path_output + 'Output\\covariance_matrix_RCNL_6.csv',index= False)
temp_covariance_7 = pd.DataFrame(RCNL7_covariance_matrix)
temp_covariance_7.to_csv(path_output + 'Output\\covariance_matrix_RCNL_7.csv',index= False)

temp_parameter_1 = pd.DataFrame(RCNL1_parameter_estimates)
temp_parameter_1.to_csv(path + 'output\\parameter_estimates_RCNL_1.csv',index= False)
temp_parameter_2 = pd.DataFrame(RCNL2_parameter_estimates)
temp_parameter_2.to_csv(path + 'output\\parameter_estimates_RCNL_2.csv',index= False)
temp_parameter_3 = pd.DataFrame(RCNL3_parameter_estimates)
temp_parameter_3.to_csv(path + 'output\\parameter_estimates_RCNL_3.csv',index= False)
temp_parameter_4 = pd.DataFrame(RCNL4_parameter_estimates)
temp_parameter_4.to_csv(path + 'output\\parameter_estimates_RCNL_4.csv',index= False)
temp_parameter_5 = pd.DataFrame(RCNL5_parameter_estimates)
temp_parameter_5.to_csv(path + 'output\\parameter_estimates_RCNL_5.csv',index= False)
temp_parameter_6 = pd.DataFrame(RCNL6_parameter_estimates)
temp_parameter_6.to_csv(path + 'output\\parameter_estimates_RCNL_6.csv',index= False)
temp_parameter_7 = pd.DataFrame(RCNL7_parameter_estimates)
temp_parameter_7.to_csv(path + 'output\\parameter_estimates_RCNL_7.csv',index= False)

In [69]:
# Output to table
table_estimates_RCNL =   [["Variable","Parameter","ECMA","Replication","(1)","(2)","(3)","(4)","(5)","(6)","(7)"],
                    ["Price","$alpha$", ecma_price_coef, matlab_price_coef, RCNL1_price_coef, RCNL2_price_coef, RCNL5_price_coef,RCNL6_price_coef, RCNL3_price_coef, RCNL4_price_coef, RCNL7_price_coef],
                    ["Price SE","", '('+str(ecma_price_se)+')', '('+str(matlab_price_se)+')', '('+str(RCNL1_price_se)+')', '('+str(RCNL2_price_se)+')', '('+str(RCNL5_price_se)+')', '('+str(RCNL6_price_se)+')', '('+str(RCNL3_price_se)+')', '('+str(RCNL4_price_se)+')', '('+str(RCNL7_price_se)+')'],
                    ["Nesting Parameter","$rho$", ecma_nesting_coef, matlab_nesting_coef, RCNL1_nesting_coef, RCNL2_nesting_coef, RCNL5_nesting_coef,RCNL6_nesting_coef, RCNL3_nesting_coef, RCNL4_nesting_coef, RCNL7_nesting_coef],
                    ["Nesting SE","", '('+str(ecma_nesting_se)+')', '('+str(matlab_nesting_se)+')',  '('+str(RCNL1_nesting_se)+')',  '('+str(RCNL2_nesting_se)+')',  '('+str(RCNL5_nesting_se)+')','('+str(RCNL6_nesting_se)+')',  '('+str(RCNL3_nesting_se)+')',  '('+str(RCNL4_nesting_se)+')',  '('+str(RCNL7_nesting_se)+')'],
                    ["Demographic Interactions"],
                    ["Income X Price","$pi_1$",ecma_income_price,matlab_income_price,RCNL1_income_price,RCNL2_income_price,RCNL5_income_price,RCNL6_income_price,RCNL3_income_price,RCNL4_income_price,RCNL7_income_price],
                    ["Income X Price SE","",'('+str(ecma_income_price_se)+')','('+str(matlab_income_price_se)+')','('+str(RCNL1_income_price_se)+')','('+str(RCNL2_income_price_se)+')','('+str(RCNL5_income_price_se)+')','('+str(RCNL6_income_price_se)+')','('+str(RCNL3_income_price_se)+')','('+str(RCNL4_income_price_se)+')','('+str(RCNL7_income_price_se)+')'],
                    ["Income X Constant","$pi_2$",ecma_income_constant,matlab_income_constant,RCNL1_income_constant,RCNL2_income_constant,RCNL5_income_constant,RCNL6_income_constant,RCNL3_income_constant,RCNL4_income_constant,RCNL7_income_constant],
                    ["Income X Constant SE","",'('+str(ecma_income_constant_se)+')','('+str(matlab_income_constant_se)+')','('+str(RCNL1_income_constant_se)+')','('+str(RCNL2_income_constant_se)+')','('+str(RCNL5_income_constant_se)+')','('+str(RCNL6_income_constant_se)+')','('+str(RCNL3_income_constant_se)+')','('+str(RCNL4_income_constant_se)+')','('+str(RCNL7_income_constant_se)+')'],      
                    ["Income X Calories","$pi_3$",ecma_income_calories,matlab_income_calories,RCNL1_income_calories,RCNL2_income_calories,RCNL5_income_calories,RCNL6_income_calories,RCNL3_income_calories,RCNL4_income_calories,RCNL7_income_calories],
                    ["Income X Calories SE","",'('+str(ecma_income_calories_se)+')','('+str(matlab_income_calories_se)+')','('+str(RCNL1_income_calories_se)+')','('+str(RCNL2_income_calories_se)+')','('+str(RCNL5_income_calories_se)+')','('+str(RCNL6_income_calories_se)+')','('+str(RCNL3_income_calories_se)+')','('+str(RCNL4_income_calories_se)+')','('+str(RCNL7_income_calories_se)+')'],                          
                    ["Other Statistics"],
                    ["Median Own Price Elasticity","",ecma_elast_own, matlab_elast_own, RCNL1_elast_own, RCNL2_elast_own, RCNL5_elast_own, RCNL6_elast_own, RCNL3_elast_own, RCNL4_elast_own, RCNL7_elast_own],
                    ["Median Market Price Elasticity","",ecma_elast_mkt,matlab_elast_mkt, RCNL1_elast_mkt, RCNL2_elast_mkt, RCNL5_elast_mkt, RCNL6_elast_mkt, RCNL3_elast_mkt, RCNL4_elast_mkt, RCNL7_elast_mkt],
                    ["Median Outside Diversion (%)","",ecma_out_diversion,matlab_out_diversion, RCNL1_out_diversion, RCNL2_out_diversion, RCNL5_out_diversion, RCNL6_out_diversion, RCNL3_out_diversion, RCNL4_out_diversion, RCNL7_out_diversion],
                    ["Jstat","",ecma_Jstat,matlab_Jstat,RCNL1_Jstat,RCNL2_Jstat,RCNL5_Jstat,RCNL6_Jstat,RCNL3_Jstat,RCNL4_Jstat,RCNL7_Jstat],
                    ["Fval","",ecma_fval,matlab_fval,RCNL1_fval,RCNL2_fval,RCNL5_fval,RCNL6_fval,RCNL3_fval,RCNL4_fval,RCNL7_fval],
                    ["Observations","",Obs,Obs,Obs,Obs,Obs,Obs,Obs,Obs,Obs],
                    ["Method","","GMM2s","GMM2s","GMM2s","GMM2s","GMM2s","GMM2s","GMM2s","GMM2s","GMM2s"],
                    ["Clustered W","","Yes","Yes","Yes","Yes","Yes","Yes","Yes","Yes","No"],
                    ["Centered Moments","","No","No","Yes","Yes","Yes","No","No","No","No"],
                    ["Optimization Routine","","Nelder-Mead","Nelder-Mead","l-bfgs-b","trust-constr","Nelder-Mead","l-bfgs-b","trust-constr","nelder-mead","l-bfgs-b"],      
                    ["Software","","Matlab","Matlab","PyBLP","PyBLP","PyBLP","PyBLP","PyBLP","PyBLP","PyBLP"],
                    ["Time (min)","","-",matlab_time,RCNL1_time,RCNL2_time,RCNL5_time,RCNL6_time,RCNL3_time,RCNL4_time,RCNL7_time]] 
table_RCNL = tabulate.tabulate(table_estimates_RCNL,headers="firstrow")
df_RCNL = pd.DataFrame(table_estimates_RCNL)
df_RCNL.to_csv(path_output + 'Output\\table_RCNL.csv',index=False)
df_RCNL

,0,1,2,3,4,5,6,7,8,9,10
0,Variable,Parameter,ECMA,Replication,(1),(2),(3),(4),(5),(6),(7)
1,Price,$alpha$,-0.0887,-0.0841,0.0002,-0.041,0.0002,-0.0833,-0.1452,-0.0834,-0.2654
2,Price SE,,(0.0141),(0.0137),(0.0116),(0.0313),(0.0116),(0.0209),(0.0263),(0.0209),(0.0871)
3,Nesting Parameter,$rho$,0.8299,0.8359,0.9602,0.6091,0.9602,0.8378,0.6847,0.8377,0.5827
4,Nesting SE,,(0.0402),(0.0389),(0.0228),(0.0902),(0.0228),(0.0384),(0.0577),(0.0384),(0.1416)
5,Demographic Interactions,None,None,None,None,None,None,None,None,None,None
6,Income X Price,$pi_1$,0.0007,0.0006,-0.0003,-0.0006,-0.0003,0.0006,0.0015,0.0006,0.0019
7,Income X Price SE,,(0.0002),(0.0002),(0.0002),(0.0016),(0.0002),(0.0002),(0.0004),(0.0002),(0.0007)
8,Income X Constant,$pi_2$,0.0143,0.0147,0.0209,0.0041,0.0209,0.0144,0.0062,0.0144,0.0044
9,Income X Constant SE,,(0.0051),(0.0049),(0.0045),(0.0143),(0.0045),(0.0049),(0.0059),(0.0049),(0.009)
